In [ ]:
'''
-Lite work in progress kring att öppna och analysera tweets.
-Notebooken måste ligga i samma mapp som all data. 
-Rekommenderar att inte köra alla celler, kan nog ta ett tag
 när vi börjar få ihop mer data.
'''

In [2]:
import json
import glob
import os
import re

In [23]:
#Printar lite extra grejs som kan vara intressant om saker går fel
DEBUG = False

In [24]:
#Genererar ett filträd. Måste ligga i samma mapp som datamapparna
# tree[date][industry][company][handle] är en lista med alla filer för den användaren (alltså: mentions, till, från).  

#TESTAD 19/3, Patrik har kontrollerat att längden på fillistan stämmer överrens med antal jsonfiler och att filnamn
# inte förekommer dubbelt. Koden är testad när notebooken ligger i samma mapp som två datamappar (2020-03-18, 2020-03-19).


tree = {}
INDUSTRIES = ["Airlines", "Online Travel Agency"]

#Lista med alla mappar i working directory som heter något i stil med 1111-11-11
collections = [d for d in glob.glob("*") if re.fullmatch(r'[0-9]{4}-[0-9]{2}-[0-9]{2}', d)]

for col in collections:
    tree[col] = {}
    for ind in INDUSTRIES:
        tree[col][ind] = {}
        
        #Den här fångar alla undermappar i /2020-xx-xx/<industry>/
        companies = [os.path.basename(x) for x in glob.glob(os.path.join(col, ind, "*"))]
        
        for comp in companies:
            tree[col][ind][comp] = {}
            
            #Traskar igenom "underfilträdet" för varje företag, alltså med /2020-xx-xx/industry/company/ som root. 
            #Lite snårig men finns bra dokumentation online.
            for root, dirs, files in os.walk(os.path.join(col, ind, comp), topdown=True):
                if dirs:
                    for handle in dirs:
                        tree[col][ind][comp][handle] = []
                else:
                    for f in files:
                        if f.find("tweet") == 0:
                            tree[col][ind][comp][os.path.basename(root)].append(os.path.join(root, f))
                            
#Printar en översikt av hela filträdet
if DEBUG:
    for c in tree.keys():
        print(c)
        for ind in tree[c].keys():
            print('\t', ind)
            for comp in tree[c][ind].keys():
                print('\t\t', comp)
                for handle in tree[c][ind][comp].keys():
                    print('\t\t\t', handle)
                    print('\t\t\t', "antal json:", str(len(tree[c][ind][comp][handle])))
                    for f in tree[c][ind][comp][handle]:
                        print('\t\t\t\t', f)

In [26]:
#Enkel tweeträknare, läser all data så kan ju ta ett tag. 
#Testad för två samlingar (2020-03-18, 2020-03-19)
counts = {}
for date in tree.keys():
    counts[date] = {}
    for industry in tree[date].keys():
        for company in tree[date][industry].keys():
            for handle in tree[date][industry][company].keys():
                counts[date][handle] = 0
                for path in tree[date][industry][company][handle]:
                    if DEBUG:
                        print(path)
                    with open(path, 'r') as f:
                        tweet_list = json.loads(f.read())
                    counts[date][handle] += len(tweet_list)
                    
print("Counts: ")
total = 0 #OBS total dubbelräknar tweets om samma tweets finns i olika samlingar. 
for date in counts.keys():
    print(date)
    for handle in counts[date].keys():
        print("\t", handle, counts[date][handle])
        total += counts[date][handle]
print("total:", total)

Counts: 
2020-03-18
	 @AerLingus 19998
	 @aeroflot 4015
	 @airfrance 6623
	 @Alitalia 7368
	 @British_Airways 29000
	 @Finnair 2593
	 @flybe 851
	 @iberia_en 2541
	 @jet2tweets 19019
	 @lufthansa 10307
	 @Fly_Norwegian 7369
	 @flymepegasus 1139
	 @Ryanair 43496
	 @SAS 4551
	 @tapairportugal 8472
	 @TurkishAirlines 7904
	 @vueling 11809
	 @easyJet 39455
	 @agoda 1658
	 @bookingcom 8508
	 @Cheapflights 5
	 @Expedia 20357
	 @ExpediaUK 3610
	 @HomeAwayUK 143
	 @homeaway 1449
	 @HotelTonight 308
	 @HotelTonightCX 168
	 @HotelTonightEU 0
	 @hotelsdotcom 0
	 @Hotwire 1617
	 @KAYAK 220
	 @lastminute_com 5555
	 @OntheBeachUK 2016
	 @Orbitz 1830
	 @priceline 4207
	 @TUIGroup 605
	 @TUIUK 14902
	 @travelocity 2880
	 @trivago 1041
	 @Vrbo 10885
2020-03-19
	 @British_Airways 46619
	 @HotelTonightEU 0
	 @hotelsdotcom 8822
total: 363915
